# Deployment Notebook

### OCR Invoice BYOP Model Deployment

This notebook serves to deploy the OCR Invoice Model.

In order to run this notebook, you must:

1. Update the `./private_key` folder with your Google service account key. Please rename it to `secret_key.json`.
2. Update the variables in the first code block to match your requirements.

If you have any questions, please reach out to the Wallaroo Team!

In [1]:
# Update the private key folder
wallaroo_api_key = "OCR_data_engineer_creds.json"

# Wallaroo Variables
workspace_name = 'realpage-ocr'  # rename 
pipeline_name = 'ocr-mega'       # rename
model_name = 'mega-byop'         # rename
model_file_name = 'mega-byop.zip' # rename

# Location of the BYOP folder
folder_to_zip = 'ocr-mega-byop'

# Start Up Tasks

These include
- Imports
- Connecting to the Wallaroo Client
- Connecting to a workspace

In [2]:
# Needed Imports
import wallaroo
from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework

import pyarrow as pa
import numpy as np
import pandas as pd
import json
import os

import zipfile

pd.set_option('display.max_columns', None)

In [3]:
# Connecting to the Wallaro Client
wl = wallaroo.Client(request_timeout=600)

Please log into the following URL in a web browser:

	https://keycloak.wallaroo.realpage.com/auth/realms/master/device?user_code=WUAF-OOHW

Login successful!


In [4]:
# Creating or connecting to a workspace
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)

{'name': 'realpage-ocr', 'id': 47, 'archived': False, 'created_by': 'eb5fb6dc-cb10-4f3d-a96e-2dcba5e0a10a', 'created_at': '2024-09-19T15:42:31.653268+00:00', 'models': [{'name': 'mega-byop', 'versions': 6, 'owner_id': '""', 'last_update_time': datetime.datetime(2024, 10, 2, 14, 9, 34, 580124, tzinfo=tzutc()), 'created_at': datetime.datetime(2024, 9, 19, 15, 45, 33, 647306, tzinfo=tzutc())}], 'pipelines': [{'name': 'ocr-mega', 'create_time': datetime.datetime(2024, 9, 19, 15, 54, 27, 301779, tzinfo=tzutc()), 'definition': '[]'}]}

# Mega OCR Deployment
## Input and Output Schema

For this model, we will be sending it a list of buffered byte files. This occurs because PDFs are read as bytes, which we will then convert to uint8s.

The output for this model is a list of strings. Each PDF will return a list of strings. The size of the list depends on the number of pages the PDF has, or the output of the model per page.


In [5]:
input_schema = pa.schema([
    pa.field("buffered_byte_files", pa.list_(pa.uint8()),nullable=False),
    pa.field("input_product_code", pa.string(),nullable=False),

])

output_schema = pa.schema([
    pa.field("generated_text", pa.list_(pa.string())),
])

# Packaging Up the BYOP Model

This step takes in the raw BYOP files and zips them up to be uploaded to Wallaroo.

In [6]:
def zip_folder_contents(folder_path, zip_file_path):
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, start=folder_path)
                zipf.write(file_path, arcname)

zip_folder_contents(folder_to_zip, model_file_name)

with zipfile.ZipFile(model_file_name, 'r') as zipf:
    print(zipf.namelist())

['ocr-mega.py', 'requirements.txt', '.ipynb_checkpoints/ocr-mega-checkpoint.py']


# Model Upload to Wallaroo

Here we upload the zip file to Wallaroo.

In [7]:
model = wl.upload_model(model_name, 
    model_file_name,
    framework=Framework.CUSTOM,
    input_schema=input_schema,
    output_schema=output_schema,
)
model

Waiting for model loading - this will take up to 10.0min.
Model is pending loading to a container runtime..
Model is attempting loading to a container runtime........successful

Ready


Name,mega-byop
Version,4cfb6519-d858-46fc-bc19-2580fdc05f4f
File Name,mega-byop.zip
SHA,0226c55f3842db621863fd473546eb23250bed76aedb42b580e24b1a2d458400
Status,ready
Image Path,proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2024.1.0-5409
Architecture,x86
Acceleration,none
Updated At,2024-03-Oct 20:18:01


In [ ]:
# to retrieve a model that has been uploaded.
# model = wl.get_model(name=model_name,version="dda4894f-064a-428b-aa38-096504d53c0a")
# model

# Defining the Deployment Config

For this model, we have found this to be a successful configuration. This also uploads the secret key to be utilized with the BYOP model.

In [8]:
deployment_config = DeploymentConfigBuilder() \
    .cpus(1).memory('8Gi') \
    .sidekick_cpus(model, 2) \
    .sidekick_memory(model, '8Gi') \
    .sidekick_env(model, {"WALLAROO_CREDENTIALS": str(json.load(open(wallaroo_api_key, 'r')))}) \
    .build()

# Creating the Pipeline

In [9]:
# Find the pipeline, if no pipeline is found, build a new one.
try:
    pipeline = wl.get_pipeline(pipeline_name)
except:
    pipeline = wl.build_pipeline(pipeline_name)

print(pipeline)

{'name': 'ocr-mega', 'create_time': datetime.datetime(2024, 9, 19, 15, 54, 27, 301779, tzinfo=tzutc()), 'definition': '[]'}


# Deploy the Pipeline

This will be used if you need to clean out the previous version of the pipeline.

Sometimes the pipeline may not deploy within the standard amount of time. If you look at the status and see "pending," please keep refreshing the status using `pipeline.status()`.

In [11]:
pipeline.undeploy()
pipeline.clear()
pipeline.add_model_step(model)
pipeline.deploy(deployment_config=deployment_config)

Waiting for undeployment - this will take up to 600s .................................. ok
Waiting for deployment - this will take up to 600s ........... ok


name,ocr-mega
created,2024-09-19 15:54:27.301779+00:00
last_updated,2024-10-03 20:18:54.038789+00:00
deployed,True
arch,x86
accel,none
tags,
versions,"9f38fe8a-de1f-430b-b622-35c027a9836e, 059f4e68-3e00-4a23-bd31-98da4bc131b8, 1fa89479-13a1-4d65-aeb0-94406e77c454, ec370fbb-0d19-4691-bc70-a301ebcb8ad3, f1a90017-06e2-45c8-a5d5-d658b414cb8a, 1d7b8b2f-14ac-4bc2-932c-4f0280787643, 80c3f80f-2469-46c5-8adb-852db066a730"
steps,mega-byop
published,False


In [12]:
print(pipeline.status())

{'status': 'Running', 'details': [], 'engines': [{'ip': '10.140.16.9', 'name': 'engine-76bbf59bfb-nsqtr', 'status': 'Running', 'reason': None, 'details': [], 'pipeline_statuses': {'pipelines': [{'id': 'ocr-mega', 'status': 'Running', 'version': '9f38fe8a-de1f-430b-b622-35c027a9836e'}]}, 'model_statuses': {'models': [{'name': 'mega-byop', 'sha': '0226c55f3842db621863fd473546eb23250bed76aedb42b580e24b1a2d458400', 'status': 'Running', 'version': '4cfb6519-d858-46fc-bc19-2580fdc05f4f'}]}}], 'engine_lbs': [{'ip': '10.140.16.8', 'name': 'engine-lb-7b5d4689fd-wrxb9', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': [{'ip': '10.140.26.15', 'name': 'engine-sidekick-mega-byop-129-867fb65978-p9brf', 'status': 'Running', 'reason': None, 'details': [], 'statuses': '\n'}]}


# Input Test Data

Here there are two examples loaded into a batch input.

We are using a two-page PDF and a one-page PDF. This is then packaged into the input dataframe used for inferencing.

Steps to process data:
- Read a PDF.
- Convert the PDF bytes to uint8s.
- Take the buffered input and insert it into the value of `buffered_byte_files`. This was the input schema we had defined earlier.


In [13]:
with open("sample_data/52807-combined.pdf", 'rb') as file:
    pdf_file_bytes = file.read()
    file.close()

with open("sample_data/k1.pdf",'rb') as file:
    pdf_file_bytes_1 = file.read()
    file.close()
    
with open("sample_data/hoa_budget.pdf",'rb') as file:
    pdf_file_bytes_2 = file.read()
    file.close()


buffered_input = np.frombuffer(pdf_file_bytes, dtype=np.uint8)
buffered_input_1 = np.frombuffer(pdf_file_bytes_1, dtype=np.uint8)
buffered_input_2 = np.frombuffer(pdf_file_bytes_2, dtype=np.uint8)


combined_df = pd.DataFrame({
        "input_product_code" : ["INV","K1","HOA"],
        "buffered_byte_files": [buffered_input.tolist(),buffered_input_1.tolist(),buffered_input_2.tolist()]
    })

# input_df = pd.DataFrame(
#     {
#         "input_product_code" : ["INV"],
#         "buffered_byte_files": [buffered_input.tolist()]
#     })

# Make an Inference

In [14]:
result = pipeline.infer(combined_df,timeout=600)
display(pd.DataFrame(result))

,time,in.buffered_byte_files,in.input_product_code,out.generated_text,anomaly.count
0,2024-10-03 20:19:22.391,"[37, 80, 68, 70, 45, 49, 46, 54, 13, 37, 226, ...",INV,"[{'INVOICE NUMBER': '52807', 'ACCOUNT NUMBER':...",0
1,2024-10-03 20:19:22.391,"[37, 80, 68, 70, 45, 49, 46, 51, 10, 37, 196, ...",K1,[{'Schedule K-1 (Form 1065) 2021': {'Part I In...,0
2,2024-10-03 20:19:22.391,"[37, 80, 68, 70, 45, 49, 46, 51, 10, 37, 226, ...",HOA,[ Category ...,0


In [15]:
print(((result["out.generated_text"][0])))

["{'INVOICE NUMBER': '52807', 'ACCOUNT NUMBER': 'LINCOLN PK', 'SALES #': None, 'DEPT. #': None, 'LOCATION': 'W', 'SOLD TO': 'RESIDENCES @ LINCOLN PK\\n1 MIDWAY PARK DR\\nN. DARTMOUTH\\n(508)-938-5139', 'SHIP TO': 'SRV LINCOLN PK', 'INVOICE DATE': datetime.date(2024, 1, 11), 'PO NUMBER': '157580', 'TERMS': 'NET DUE', 'PAY METHOD': 'ON ACCOUNT', 'SUB TOTAL': 389.88, 'TAX': '14.99', 'DELIVERY': None, 'DISCOUNT': None, 'TOTAL': 404.87, 'LineItems': [{'QTY': '1', 'MAKE': 'LAB', 'PRODUCT': 'SVC', 'DESCRIPTION': 'SERVICE & DIAGNOSTIC', 'PRICE': '150.00', 'EXTENSION': '150.00'}, {'QTY': '1', 'MAKE': 'WCI', 'PRODUCT': '316405000', 'DESCRIPTION': 'SEAL', 'PRICE': '136.98', 'EXTENSION': '136.98'}, {'QTY': '1', 'MAKE': 'WCI', 'PRODUCT': '316575500', 'DESCRIPTION': 'BEZEL', 'PRICE': '102.90', 'EXTENSION': '102.90'}], 'Notes': '*APT 207*\\nORDERED AND REPLACED SEAL AND BEZEL\\nALL OK NOW COMPLETE'}", "{'INVOICE': '63170', 'DATE': datetime.date(2024, 1, 13), 'ACCT': '33477', 'EMPLID': 'LXHOUTMA', 'So

In [16]:
print(((result["out.generated_text"][1])))

['{\'Schedule K-1 (Form 1065) 2021\': {\'Part I Information About the Partnership\': {"A Partnership\'s employer identification number": \'46-4097730\', "B Partnership\'s name, address, city, state, and ZIP code": \'ENVIVA PARTNERS, LP\\n7200 WISCONSIN AVENUE\\nSUITE 1000\\nBETHESDA, MD 20814\', \'C IRS Center where partnership filed return\': \'e-file\', \'D Check if this is a publicly traded partnership (PTP)\': \'BLANK\'}, \'Part II Information About the Partner\': {"E Partner\'s SSN or TIN (Do not use TIN of a disregarded entity. See instructions.)": \'----7730\', "F Partner\'s name, address, city, state, and ZIP code": \'SAMPLE K-1\\n10000 UNITS\\n7200 WISCONSIN AVENUE\\nSUITE 1000\\nBETHESDA, MD 20814\', \'G General partner or LLC member-manager\': \'BLANK\', \'Limited partner or other LLC member\': \'X\', \'H1 Domestic partner\': \'X\', \'Foreign partner\': \'BLANK\', "H2 If the partner is a disregarded entity (DE), enter the partner\'s": {\'TIN\': \'BLANK\', \'Name\': \'BLANK\'

In [17]:
print(((result["out.generated_text"][2])))

['              Category                         Account Month    Amount\n0               INCOME             3000 Homeowner Fees   Jan  21082.26\n1               INCOME             3000 Homeowner Fees   Feb  21082.26\n2               INCOME             3000 Homeowner Fees   Mar  21082.26\n3               INCOME             3000 Homeowner Fees   Apr  21082.26\n4               INCOME             3000 Homeowner Fees   May  21082.26\n5               INCOME             3000 Homeowner Fees   Jun  21082.26\n6               INCOME             3000 Homeowner Fees   Jul  21082.26\n7               INCOME             3000 Homeowner Fees   Aug  21082.26\n8               INCOME             3000 Homeowner Fees   Sep  21082.26\n9               INCOME             3000 Homeowner Fees   Oct  21082.26\n10              INCOME             3000 Homeowner Fees   Nov  21082.26\n11              INCOME             3000 Homeowner Fees   Dec  21082.30\n12             EXPENSE                  4010 Insurance   Jan  

In [37]:
import ast
import re
import pandas as pd

# Function to replace datetime.date with date strings
def replace_datetime_date(s):
    pattern = r'datetime\.date\((\d+), (\d+), (\d+)\)'
    def repl(match):
        year, month, day = match.groups()
        return f"'{year}-{month.zfill(2)}-{day.zfill(2)}'"
    return re.sub(pattern, repl, s)

# Process the data
processed_data = [replace_datetime_date(s) for s in result["out.generated_text"][0]]

# Parse the strings into dictionaries
parsed_data = [ast.literal_eval(s) for s in processed_data]

# List to hold combined invoice and line item data
combined_data = []

for invoice in parsed_data:
    # Copy invoice data
    invoice_data = invoice.copy()
    # Determine line items key
    if 'line_items' in invoice_data:
        line_items_key = 'line_items'
    elif 'ITEM' in invoice_data:
        line_items_key = 'ITEM'
    else:
        line_items_key = None
    # Extract line items
    if line_items_key:
        line_items = invoice_data.pop(line_items_key)
        # For each line item, combine with invoice data
        for item in line_items:
            combined_row = invoice_data.copy()
            combined_row.update(item)
            combined_data.append(combined_row)
    else:
        # No line items, just add the invoice data
        combined_data.append(invoice_data)

# Create DataFrame
df = pd.DataFrame(combined_data)

df


,invoice_number,invoice_date,account_number,vendor,vendor_address,vendor_phone,customer_name,customer_address,customer_phone,ship_to,ship_to_id,po_number,terms,pay_method,subtotal,tax,delivery,discount,total,notes,qty,make,product,description,price,extension,INVOICE,DATE,ACCT,EMPLID,Sold To:,SUBTOTAL,Tax,TOTAL,PAYMENTS,BALANCE,QTY,ITEM,DESCRIPTION,PRICE EA
0,52807,2024-01-11,LINCOLN PK,APPLIANCE CARE SERVICE COMPANY,"1261 GLOBE STREET\nFALL RIVER, MA 02721",508-674-0361,RESIDENCES @ LINCOLN PK,1 MIDWAY PARK DR\nN.DARTMOUTH\nMA 02747,(508)-938-5139,SRV LINCOLN PK,157580,157580,NET DUE,ON ACCOUNT,389.88,14.99,NaN,NaN,404.87,*APT 207*\nORDERED AND REPLACED SEAL AND BEZEL...,1,LAB,SVC,SERVICE & DIAGNOSTIC,150.00,150.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,52807,2024-01-11,LINCOLN PK,APPLIANCE CARE SERVICE COMPANY,"1261 GLOBE STREET\nFALL RIVER, MA 02721",508-674-0361,RESIDENCES @ LINCOLN PK,1 MIDWAY PARK DR\nN.DARTMOUTH\nMA 02747,(508)-938-5139,SRV LINCOLN PK,157580,157580,NET DUE,ON ACCOUNT,389.88,14.99,NaN,NaN,404.87,*APT 207*\nORDERED AND REPLACED SEAL AND BEZEL...,1,WCI,316405000,SEAL,136.98,136.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52807,2024-01-11,LINCOLN PK,APPLIANCE CARE SERVICE COMPANY,"1261 GLOBE STREET\nFALL RIVER, MA 02721",508-674-0361,RESIDENCES @ LINCOLN PK,1 MIDWAY PARK DR\nN.DARTMOUTH\nMA 02747,(508)-938-5139,SRV LINCOLN PK,157580,157580,NET DUE,ON ACCOUNT,389.88,14.99,NaN,NaN,404.87,*APT 207*\nORDERED AND REPLACED SEAL AND BEZEL...,1,WCI,316575500,BEZEL,102.90,102.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,63170,2024-01-13,33477,LXHOUTMA,MARION VILLAGE ESTATES\n32 VILLAGE DRIVE\nMARI...,139.95,8.75,139.95,148.70,148.70,1,320023,PUSH BUTTON 1.1 CU.FT BLACK,139.95


In [39]:
import ast
import pandas as pd

# Parse the string into a dictionary
parsed_data = [ast.literal_eval(entry) for entry in result["out.generated_text"][1]]

# Function to flatten nested dictionaries
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Flatten the data
flattened_data = []
for entry in parsed_data:
    for key, value in entry.items():
        flattened_entry = flatten_dict(value)
        flattened_entry['Form'] = key
        flattened_data.append(flattened_entry)

# Create DataFrame
k1_df = pd.DataFrame(flattened_data)

k1_df



,Part I Information About the Partnership_A Partnership's employer identification number,"Part I Information About the Partnership_B Partnership's name, address, city, state, and ZIP code",Part I Information About the Partnership_C IRS Center where partnership filed return,Part I Information About the Partnership_D Check if this is a publicly traded partnership (PTP),Part II Information About the Partner_E Partner's SSN or TIN (Do not use TIN of a disregarded entity. See instructions.),"Part II Information About the Partner_F Partner's name, address, city, state, and ZIP code",Part II Information About the Partner_G General partner or LLC member-manager,Part II Information About the Partner_Limited partner or other LLC member,Part II Information About the Partner_H1 Domestic partner,Part II Information About the Partner_Foreign partner,"Part II Information About the Partner_H2 If the partner is a disregarded entity (DE), enter the partner's_TIN","Part II Information About the Partner_H2 If the partner is a disregarded entity (DE), enter the partner's_Name",Part II Information About the Partner_I1 What type of entity is this partner?,"Part II Information About the Partner_I2 If this partner is a retirement plan (IRA/SEP/Keogh/etc.), check here","J Partner's share of profit, loss, and capital_Profit_Beginning","J Partner's share of profit, loss, and capital_Profit_Ending","J Partner's share of profit, loss, and capital_Loss_Beginning","J Partner's share of profit, loss, and capital_Loss_Ending","J Partner's share of profit, loss, and capital_Capital_Beginning","J Partner's share of profit, loss, and capital_Capital_Ending",K Partner's share of liabilities_Nonrecourse_Beginning,K Partner's share of liabilities_Nonrecourse_Ending,K Partner's share of liabilities_Qualified nonrecourse financing_Beginning,K Partner's share of liabilities_Qualified nonrecourse financing_Ending,K Partner's share of liabilities_Recourse_Beginning,K Partner's share of liabilities_Recourse_Ending,L Partner's capital account analysis_Beginning capital account,L Partner's capital account analysis_Capital contributed during the year,L Partner's capital account analysis_Current year net income (loss),L Partner's capital account analysis_Other increase (decrease) (attach explanation),L Partner's capital account analysis_Withdrawals & distributions,L Partner's capital account analysis_Ending capital account,M Did the partner contribute property with a built-in gain or loss?,N Partner's Share of Net Unrecognized Section 704(c) Gain or (Loss)_Beginning,N Partner's Share of Net Unrecognized Section 704(c) Gain or (Loss)_Ending,N Partner's Share of Net Unrecognized Section 704(c) Gain or (Loss)_Final K-1,N Partner's Share of Net Unrecognized Section 704(c) Gain or (Loss)_Amended K-1,"Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_1 Ordinary business income (loss)","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_2 Net rental real estate income (loss)","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_3 Other net rental income (loss)","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_4a Guaranteed payments for services","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_4b Guaranteed payments for capital","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_4c Total guaranteed payments","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_5 Interest income","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_6a Ordinary dividends","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_6b Qualified dividends","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_6c Dividend equivalents","Part III Partner's Share of Cu

## Next Step: Utilize the API